In [1]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import numpy as np
import joblib
import os
from catboost import CatBoostRegressor

# Initialize the main window
root = tk.Tk()
root.title("GUI model for Predicting Concrete CS Based on 7 Ingredients & Curing Age")

# Create the main canvas
canvas = tk.Canvas(root, width=1000, height=650, background='#FFFFFF')
canvas.pack()

# Load the image from the desktop and create an image on the canvas using PIL
image_path = "C:/Users/asus1/Desktop/image.jpg"

try:
    image = Image.open(image_path)
except FileNotFoundError:
    messagebox.showerror("Error", f"The file at {image_path} was not found. Please check the path and try again.")
    root.destroy()
    exit()

# Scale the image to fit the canvas if needed
image = image.resize((1000, 200), Image.LANCZOS)

# Store the image in an attribute of root to prevent garbage collection
root.image_storage = ImageTk.PhotoImage(image)
canvas.create_image(500, 200, image=root.image_storage, anchor="center")

# Function to create labels on the canvas
def create_label(text, font, fg, bg, x, y):
    label = tk.Label(root, text=text, font=font, fg=fg, bg=bg)
    canvas.create_window(x, y, anchor="w", window=label)
    return label

# Check if the CatBoost model file exists
model_path_catb = "C:/Users/asus1/Desktop/CGB.joblib"
if not os.path.exists(model_path_catb):
    messagebox.showerror("Error", "CatBoost model file not found.")
    root.destroy()
    exit()

# Load the CatBoost model
model_catb = joblib.load(model_path_catb)

# Define the GUI elements
label_inputdefinetitle2 = create_label('GUI model for Predicting Concrete CS Based on 7 Ingredients & Curing Age', ('Comic Sans MS', 18, 'bold', 'underline'), '#0000FF', '#FFFF00', 20,30)
label_inputdefinetitle3 = create_label('Developed by: Mohamed K. Elshaarawy, Abdelrahman K. Hamed & Mostafa M. Alsaadawi', ('Comic Sans MS', 14, 'bold'), '#C00000', '#FFFFFF', 20,70)
label_inputdefinetitle = create_label('Definition of Parameters', ('Comic Sans MS', 16, 'bold'), '#000000', '#FFFFFF', 50, 310)

param_labels = [
    'X1: Cement (kg/m^3)',
    'X2: Blast Furnace Slag (kg/m^3)',
    'X3: Fly Ash (kg/m^3)',
    'X4: Water (kg/m^3)',
    'X5: Superplasticizer (kg/m^3)',
    'X6: Coarse Aggregate (kg/m^3)',
    'X7: Fine Aggregate (kg/m^3)',
    'X8: Age (day)',
]
for i, text in enumerate(param_labels, start=1):
    create_label(text, ('Comic Sans MS', 14), '#00008B', '#FFFFFF', 50, 320 + i*30)

label_inputs = create_label('Inputs', ('Comic Sans MS', 16, 'bold'), '#000000', '#FFFFFF', 620, 310)

entry_fields = ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8']
entries = {}
for i, field in enumerate(entry_fields, start=1):
    create_label(f'{field} = ', ('Comic Sans MS', 14, 'bold'), '#006600', '#FFFFFF', 550, 320 + i*30)
    entry = tk.Entry(root, font=('Comic Sans MS', 14, 'bold'), bg='#F0F0F0', highlightbackground='#000000', highlightthickness=1, bd=0, width=20)
    canvas.create_window(750, 320 + i*30, window=entry)
    entries[field] = entry

label_output = create_label('Output:', ('Comic Sans MS', 16, 'bold'), '#000000', '#FFFFFF', 50, 610)
label_result = create_label('Concrete compressive strength (MPa)', ('Comic Sans MS', 16, 'bold'), '#C00000', '#FFFFFF', 200, 610)

# Predict button command function
def predict():
    # Collect input data
    input_values = []
    for field in entry_fields:
        try:
            value = float(entries[field].get())
            if value <= 0:
                raise ValueError("Value must be positive.")
            input_values.append(value)
        except ValueError as e:
            messagebox.showerror("Error", f"Invalid input for {field}. Please enter a positive numeric value.")
            return

    input_data = np.array([input_values])
    try:
        prediction = model_catb.predict(input_data)
        label_result['text'] = f'CS= {prediction[0]:.4f} (MPa)'
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred during prediction: {e}")

# Predict button
predict_button = tk.Button(root, text='Calculate', font=('Comic Sans MS', 16), command=predict, fg='#FFFFFF', bg='dark blue')
canvas.create_window(750, 605, window=predict_button)

# Run the main loop
root.mainloop()
